In [1]:
from importlib.util import find_spec
if find_spec("model") is None:
    import sys
    sys.path.append('..')

In [2]:
import torch

In [3]:
from model._model import VGGBase, AuxiliaryConvolutions
from data_loader.data_loaders import PascalVOCDataLoader

In [4]:
base = VGGBase()


Loaded base model.



In [5]:
dl = PascalVOCDataLoader('/home/kmh/Documents/datasets/pascal', 8)

In [6]:
batch = next(iter(dl))

In [8]:
conv_4_feat, conv_7_feat = base(batch[0])

In [9]:
model = AuxiliaryConvolutions()

In [17]:
torch.onnx.export(model, conv_7_feat, './models/aux_model.onnx', input_names=["conv7_feats"], output_names=["conv8_feats", "conv9_feats", "conv10_feats", "conv11_feats"], do_constant_folding=False, verbose=True)

graph(%conv7_feats : Float(8:369664, 1024:361, 19:19, 19:1),
      %conv8_1.weight : Float(256:1024, 1024:1, 1:1, 1:1),
      %conv8_1.bias : Float(256:1),
      %conv8_2.weight : Float(512:2304, 256:9, 3:3, 3:1),
      %conv8_2.bias : Float(512:1),
      %conv9_1.weight : Float(128:512, 512:1, 1:1, 1:1),
      %conv9_1.bias : Float(128:1),
      %conv9_2.weight : Float(256:1152, 128:9, 3:3, 3:1),
      %conv9_2.bias : Float(256:1),
      %conv10_1.weight : Float(128:256, 256:1, 1:1, 1:1),
      %conv10_1.bias : Float(128:1),
      %conv10_2.weight : Float(256:1152, 128:9, 3:3, 3:1),
      %conv10_2.bias : Float(256:1),
      %conv11_1.weight : Float(128:256, 256:1, 1:1, 1:1),
      %conv11_1.bias : Float(128:1),
      %conv11_2.weight : Float(256:1152, 128:9, 3:3, 3:1),
      %conv11_2.bias : Float(256:1)):
  %17 : Float(8:92416, 256:361, 19:19, 19:1) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%conv7_feats, %conv8_1.weight, %conv8_1

../model/_model.py:165: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert conv8_2_feats.shape == (bs, 512, 10, 10)
../model/_model.py:170: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert conv9_2_feats.shape == (bs, 256, 5, 5)
../model/_model.py:175: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert conv10_2_feats.shape == (b

In [18]:
conv8_feats, conv9_feats, conv10_feats, conv11_feats = model(conv_7_feat)

In [19]:
from model._model import PredictionConvolutions

In [20]:
pred = PredictionConvolutions(n_classes=21)

In [22]:
torch.onnx.export(pred, (conv_4_feat, conv_7_feat, conv8_feats, conv9_feats, conv10_feats, conv11_feats), './models/pred_model.onnx', input_names=["conv4_feats", "conv7_feats", "conv8_feats", "conv9_feats", "conv10_feats", "conv11_feats"], output_names=["locs", "class_scores"], do_constant_folding=False, verbose=True)

graph(%conv4_feats : Float(8:739328, 512:1444, 38:38, 38:1),
      %conv7_feats : Float(8:369664, 1024:361, 19:19, 19:1),
      %conv8_feats : Float(8:51200, 512:100, 10:10, 10:1),
      %conv9_feats : Float(8:6400, 256:25, 5:5, 5:1),
      %conv10_feats : Float(8:2304, 256:9, 3:3, 3:1),
      %conv11_feats : Float(8:256, 256:1, 1:1, 1:1),
      %loc_conv4_3.weight : Float(16:4608, 512:9, 3:3, 3:1),
      %loc_conv4_3.bias : Float(16:1),
      %loc_conv7.weight : Float(24:9216, 1024:9, 3:3, 3:1),
      %loc_conv7.bias : Float(24:1),
      %loc_conv8_2.weight : Float(24:4608, 512:9, 3:3, 3:1),
      %loc_conv8_2.bias : Float(24:1),
      %loc_conv9_2.weight : Float(24:2304, 256:9, 3:3, 3:1),
      %loc_conv9_2.bias : Float(24:1),
      %loc_conv10_2.weight : Float(16:2304, 256:9, 3:3, 3:1),
      %loc_conv10_2.bias : Float(16:1),
      %loc_conv11_2.weight : Float(16:2304, 256:9, 3:3, 3:1),
      %loc_conv11_2.bias : Float(16:1),
      %cl_conv4_3.weight : Float(84:4608, 512:9, 3:3, 3:1